In [1]:
from astropy.io import fits
from astropy.table import Table,join
import numpy as np
import pylab as plt
import random
from scipy import stats
from sklearn.neighbors import KDTree
import time
from sklearn.metrics import mean_squared_error
from desitarget.targetmask import desi_mask, bgs_mask, mws_mask

In [2]:
## DATA ##
## I'm combining fastphot,fastspect, and ztile to make sure I use the same data everywhere ##

zall_path="/project/projectdirs/desi/spectro/redux/everest/zcatalog/ztile-main-bright-cumulative.fits"
data1=Table.read(zall_path,hdu=1)
needed1=["TARGETID","BGS_TARGET","SPECTYPE","DELTACHI2","Z","ZWARN"]

fastspec_path = "/project/projectdirs/desi/spectro/fastspecfit/everest/catalogs/fastspec-everest-main-bright.fits"
data2=Table.read(fastspec_path,hdu=1)
needed2=["TARGETID","OII_3726_EW","OII_3729_EW","HGAMMA_EW","HBETA_EW","OIII_4959_EW","OIII_5007_EW","NII_6548_EW","HALPHA_EW","NII_6584_EW","SII_6716_EW","SII_6731_EW",\
        "OII_3726_EW_IVAR","OII_3729_EW_IVAR","HGAMMA_EW_IVAR","HBETA_EW_IVAR","OIII_4959_EW_IVAR","OIII_5007_EW_IVAR","NII_6548_EW_IVAR","HALPHA_EW_IVAR","NII_6584_EW_IVAR","SII_6716_EW_IVAR","SII_6731_EW_IVAR"]

file_path = "/project/projectdirs/desi/spectro/fastspecfit/everest/catalogs/fastphot-everest-main-bright.fits"
data3=Table.read(file_path,hdu=1)
needed3=["TARGETID","ABSMAG_SDSS_U","ABSMAG_SDSS_G","ABSMAG_SDSS_R","ABSMAG_SDSS_I","ABSMAG_SDSS_Z"]

data4=join(data1[needed1],data2[needed2],keys="TARGETID")
data=join(data4,data3[needed3],keys="TARGETID")

## Adding the sum of OII doublets to use them as a single line
data.add_column(data["OII_3726_EW"]+data["OII_3729_EW"],name='OII_DOUBLET_EW')
data.add_column(1/(data["OII_3726_EW_IVAR"]+data["OII_3729_EW_IVAR"]),name='OII_DOUBLET_EW_IVAR')

/tmp/ipykernel_33239/4143185914.py:22: RuntimeWarning: divide by zero encountered in true_divide
  data.add_column(1/(data["OII_3726_EW_IVAR"]+data["OII_3729_EW_IVAR"]),name='OII_DOUBLET_EW_IVAR')


In [3]:
def LLR_inverse_distance(x1,x1_train,y1_train,nn):
    '''
    Local linear regression with inverse distance weight and nn number of nearest neighbors.
    
    Input
    -----
    
    x1: Matrix of features in the shape (number of data points, number of features). Outcome will be evaluated at these points.
    
    x1_train: Matrix of features used for training in the shape of (number of data points, number of features).
    
    y1_train: Matrix of outcomes used for training in the shape of (number of data points, number of outcomes).
    
    nn: Number of nearest neighbors to include for each point.
    
    
    Output
    ------
    
    EW_fit: Matrix of outcomes predicte in the same shape as y1_train.
    
    zeros: indices corresponding to points in x1 that have nearest neighbor at zero distance.
    
    '''
    
    nl=nn
    tree=KDTree(x1_train[:,:])
    dist, ind=tree.query(x1[:,:],k=nl)

    # removing points on top of each other
    zeros=np.where(dist==0)[0]
    print(zeros)
    dist=np.delete(dist,obj=zeros,axis=0)
    ind=np.delete(ind,obj=zeros,axis=0)
    x1=np.delete(x1,obj=zeros,axis=0)

    n_valid=x1.shape[0]

    theta=np.zeros([n_valid,x1.shape[1],1])
    W=np.zeros([n_valid,nl,nl])
    X=np.zeros([n_valid,nl,x1.shape[1]])
    Y=np.zeros([n_valid,nl,1])
    for j in range(nl):
        W[:,j,j]=1/dist[:,j]
        X[:,j,:]=x1_train[ind[:,j],:]
        Y[:,j,0]=y1_train[ind[:,j]]
    a1=np.zeros([n_valid,x1.shape[1],1])
    a2=np.zeros([n_valid,x1.shape[1],x1.shape[1]])
    EW_fit=np.zeros(n_valid)
    for ii in range(n_valid):
        a1[ii,:,:]=np.matmul(X[ii,:,:].transpose(),np.matmul(W[ii,:,:],Y[ii,:,:]))
        a2[ii,:,:]=np.matmul(X[ii,:,:].transpose(),np.matmul(W[ii,:,:],X[ii,:,:]))
        theta[ii,:,:]=np.matmul(np.linalg.inv(a2[ii,:,:]),a1[ii,:,:])
        EW_fit[ii]=np.matmul(theta[ii,:,:].transpose(),x_valid[ii,:])
        
    return EW_fit,zeros

In [7]:
## Selecting data and doing LLR to predict lines ##
lines=["OII_DOUBLET_EW","HGAMMA_EW","HBETA_EW","OIII_4959_EW","OIII_5007_EW","NII_6548_EW","HALPHA_EW"\
       ,"NII_6584_EW","SII_6716_EW","SII_6731_EW"]

magnitude_names=["ABSMAG_SDSS_U","ABSMAG_SDSS_G","ABSMAG_SDSS_R","ABSMAG_SDSS_I","ABSMAG_SDSS_Z"]
 
N=len(data["TARGETID"])
snr_cut=1 # signal to noise ratio cut
n=30*10**3 # initial selection size. Should be smaller later after selecting flux>0 from raw data to make sure same data is used.

# calculating snr for all lines and setting the snr cut as select_snr
snr_all=np.zeros([N,len(lines)])
snr_all[:,0]=data[lines[0]]*np.sqrt(data[lines[0]+"_IVAR"])

cut_Halpha=True
cut_all=False
for i in range(1,len(lines)):
    snr_all[:,i]=data[lines[i]]*np.sqrt(data[lines[i]+"_IVAR"])
    if cut_all:
        select_snr=select_snr*(snr_all[:,i]>snr_cut)
if cut_Halpha:
    select_snr=snr_all[:,6]>snr_cut

# calculating minimum redshift to have de-redshifted wavelengths be in the interval 3400,7000 A
w1=3400
w_min=3600
z_min=w_min/w1-1

# getting flux cut from everest target selection to make sure same data is used
run=2
select_fluxes=np.load("/global/homes/a/ashodkh/results/select_positive_fluxes_"+str(run)+".txt.npz")["arr_0"]
for l in range(1):
    l=6
    # initial target selection that doesn't include flux cut
    select=((data["BGS_TARGET"] & bgs_mask.mask("BGS_BRIGHT"))>0)*(data["SPECTYPE"]=="GALAXY")*(data["DELTACHI2"]>=25)\
        *(data["Z"]>z_min)*(data["Z"]<0.3)*(data["ZWARN"]==0)*select_snr
    target_ids=data["TARGETID"][select][:n]
    print(len(np.where(select==True)[0]))
    target_pos=np.where(select==True)[0][:n] 
    # flux cut after initial target selection and taking first n data
    n=25*10**3
    target_pos=target_pos[select_fluxes][:n]
    target_ids=target_ids[select_fluxes]
    np.savetxt("/global/homes/a/ashodkh/results/target_ids_test1.txt", target_ids[:n])
    # assigning features as colors and standardizing them. I also add ones to include the y-intercept as part of the parameter matrix
    magnitudes_s=data[magnitude_names][target_pos]  
    magnitudes=np.zeros([n,len(magnitude_names)])
    for j in range(len(magnitude_names)):
        magnitudes[:,j]=magnitudes_s[magnitude_names[j]][:n]

    ones=np.ones([n,1])
    x=np.zeros([n,len(magnitude_names)-1])
    for i in range(n):
        for j in range(len(magnitude_names)-1):
            x[i,j]=magnitudes[i,j]-magnitudes[i,j+1]
    x=np.concatenate((ones,x),axis=1)

    av_x=np.zeros(x.shape[1]-1)
    std_x=np.zeros(x.shape[1]-1)
    for i in range(1,x.shape[1]):
        av_x[i-1]=np.average(x[:,i])
        std_x[i-1]=np.std(x[:,i])
        x[:,i]=(x[:,i]-av_x[i-1])/std_x[i-1]
    
    # assigning outcomes as EW (equivalent width) and getting their inverse variance
    EW=np.log10(data[lines[l]][target_pos])
    ivar=data[lines[l]+"_IVAR"][target_pos]
    
    ## doing cross-validation by splitting data into N_cv intervals. I store all the outcomes in EW_fit_all, ivar_all, etc...
    N_cv=10
    x_split=np.split(x,N_cv)
    EW_split=np.split(EW,N_cv)
    ivar_split=np.split(ivar,N_cv)
    EW_fit_all=[]
    EW_obs_all=[]
    ivar_all=[]
    spearman_all=[]
    rms_all=[]
    nmad_all=[]
    nmad2_all=[]
    for i in range(N_cv):
        ## assigning the training and validation sets
        x_valid=x_split[i]
        EW_valid=EW_split[i]
        ivar_valid=ivar_split[i]
        x_to_combine=[]
        EW_to_combine=[]
        for j in range(N_cv):
            if j!=i:
                x_to_combine.append(x_split[j])
                EW_to_combine.append(EW_split[j])
        x_train=np.concatenate(tuple(x_to_combine),axis=0)
        EW_train=np.concatenate(tuple(EW_to_combine),axis=0)
        
        # predicting EWs using LLR
        EW_fit,zeros=LLR_inverse_distance(x_valid,x_train,EW_train,100)
        
        # removing points that are on top of each other from y_valid and its ivar
        EW_valid=np.delete(EW_valid,obj=zeros,axis=0)
        ivar_valid=np.delete(ivar_valid,obj=zeros,axis=0)
        
        # calculating spearman coefficient and nmad for fit. nmad2 has the error in it.
        nmad=np.abs(EW_fit-EW_valid)
        nmad2=np.abs(EW_fit-EW_valid)*np.sqrt(ivar_valid)*EW_valid

        EW_fit_all.append(EW_fit)
        EW_obs_all.append(EW_valid)
        ivar_all.append(ivar_valid)
        spearman_all.append(stats.spearmanr(EW_fit,EW_valid)[0])
        rms_all.append(np.sqrt(mean_squared_error(EW_fit,EW_valid)))
        nmad_all.append(1.48*np.median(nmad))
        nmad2_all.append(1.48*np.median(nmad2))

    print(lines[l])
    print(spearman_all)
    print(np.average(spearman_all))
    # print(rms_all)
    # print(np.average(rms_all))
    print(nmad_all)
    print(np.average(nmad_all))
    print(nmad2_all)
    print(np.average(nmad2_all))
    print("\n")

    ## saving data
    
    
    # #with open("/global/homes/a/ashodkh/results/logEW_fit_all_nl300_exp"+str(alpha)+".txt",'w') as f:
    # with open("/global/homes/a/ashodkh/results/logEW_fit_snr"+str(snr_cut)+"_all_nl"+str(nl)+"_distance"+lines[l]+".txt",'w') as f:    
    #     for i in range(N_cv):
    #         for j in range(len(EW_fit_all[i])-1):
    #             f.write(str(np.round(EW_fit_all[i][j],decimals=5))+',')
    #         f.write(str(np.round(EW_fit_all[i][-1],decimals=5)))
    #         f.write('\n')
    # #with open("/global/homes/a/ashodkh/results/logEW_obs_all_nl300_exp"+str(alpha)+".txt",'w') as f:
    # with open("/global/homes/a/ashodkh/results/logEW_obs_snr"+str(snr_cut)+"_all_nl"+str(nl)+"_distance"+lines[l]+".txt",'w') as f: 
    #     for i in range(N_cv):
    #         for j in range(len(EW_obs_all[i])-1):
    #             f.write(str(np.round(EW_obs_all[i][j],decimals=5))+',')
    #         f.write(str(np.round(EW_obs_all[i][-1],decimals=5)))
    #         f.write('\n')
    # with open("/global/homes/a/ashodkh/results/logEW_snr"+str(snr_cut)+"_ivar"+lines[l]+".txt",'w') as f: 
    #     for i in range(N_cv):
    #         for j in range(len(ivar_all[i])-1):
    #             f.write(str(np.round(ivar_all[i][j],decimals=5))+',')
    #         f.write(str(np.round(ivar_all[i][-1],decimals=5)))
    #         f.write('\n')

    
    # np.savez_compressed("/global/homes/a/ashodkh/results/logEW_fit_classical_selection"+str(run)+"_line"+str(lines[l])+".txt",EW_fit_all)
    # np.savez_compressed("/global/homes/a/ashodkh/results/logEW_obs_classical_selection"+str(run)+"_line"+str(lines[l])+".txt",EW_obs_all)
    # np.savez_compressed("/global/homes/a/ashodkh/results/logEW_ivar_classical_selection"+str(run)+"_line"+str(lines[l])+".txt",ivar_all)

/tmp/ipykernel_33239/3971532732.py:13: RuntimeWarning: invalid value encountered in multiply
  snr_all[:,0]=data[lines[0]]*np.sqrt(data[lines[0]+"_IVAR"])


450905
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
HALPHA_EW
[0.8430676459308234, 0.8533380708700914, 0.8506669272747084, 0.8502175596508097, 0.8411498582319775, 0.8447751198680193, 0.8375019292331092, 0.8431041428326629, 0.8598901625424261, 0.8681872931246588]
0.8491898709559287
[0.21913059181935426, 0.22943219019237532, 0.21771232796662662, 0.21791980236789607, 0.22535438173943828, 0.2191766745266001, 0.2117075073402828, 0.21807189194944449, 0.2196206799078542, 0.21246111322130576]
0.2190587161031178
[0.18353955126947796, 0.18806804097176347, 0.1795397731021607, 0.19006909580457879, 0.18901691453981287, 0.18478190765825073, 0.18190642388472575, 0.1821542870207031, 0.17983681445474778, 0.1820495402295734]
0.18409623489357946




In [5]:
# plt.plot(EW_obs_all[0],EW_fit_all[0],'*',alpha=0.1)
# plt.plot(np.arange(0,4,0.1),np.arange(0,4,0.1))
# # plt.xlim((0,4))
# plt.ylim((0,4))

# 